In [1]:
from tensorflow.keras import models, layers, optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.applications import VGG16
import numpy as np
import matplotlib.pyplot as plt
import os
os.environ['KMP_DUPLICATE_LIB_OK'] = '1'

In [2]:
BATCH_SIZE = 20

train_datagen = ImageDataGenerator(
    rescale=1./255,
    horizontal_flip=True,
    width_shift_range=0.1,
    height_shift_range=0.1,
)

train_generator = train_datagen.flow_from_directory(
    'data/dogs-vs-cats/train',
    target_size=(150, 150),
    batch_size=BATCH_SIZE,
    class_mode='binary'
)

test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
    'data/dogs-vs-cats/test',
    target_size=(150, 150),
    batch_size=BATCH_SIZE,
    class_mode='binary'
)

Found 20000 images belonging to 2 classes.
Found 5000 images belonging to 2 classes.


In [3]:
transfer_model = VGG16(weights='imagenet', include_top=False,
                      input_shape=(150, 150, 3))

transfer_model.trainable = False
transfer_model.summary()

58889256/58889256 [==============================] - 3s 0us/step
Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 150, 150, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 150, 150, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 150, 150, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 75, 75, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 75, 75, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 75, 75, 128)       147584    
                                                              

In [4]:
finetune_model = models.Sequential()
finetune_model.add(transfer_model)
finetune_model.add(layers.Flatten())
finetune_model.add(layers.Dense(512, activation='relu'))
finetune_model.add(layers.Dropout(0.5))
finetune_model.add(layers.Dense(1, activation='sigmoid'))
finetune_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 4, 4, 512)         14714688  
                                                                 
 flatten (Flatten)           (None, 8192)              0         
                                                                 
 dense (Dense)               (None, 512)               4194816   
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 1)                 513       
                                                                 
Total params: 18,910,017
Trainable params: 4,195,329
Non-trainable params: 14,714,688
_________________________________________________________________


In [5]:
finetune_model.compile(loss='binary_crossentropy',
            optimizer=optimizers.Adam(learning_rate=0.0002),
             metrics=['accuracy'])

In [6]:
early_stopping = EarlyStopping(monitor='val_loss', patience=10)

modelpath = 'model/dogs-vs-cats.hdf5'

checkpoint = ModelCheckpoint(filepath=modelpath, verbose=0, monitor='val_loss',
                             save_best_only=True)

In [ ]:
h = finetune_model.fit(train_generator, epochs=30, validation_data=test_generator,
             callbacks=[checkpoint, early_stopping])

Epoch 1/30
1000/1000 [==============================] - 629s 629ms/step - loss: 0.3228 - accuracy: 0.8565 - val_loss: 0.2321 - val_accuracy: 0.9000
Epoch 2/30
1000/1000 [==============================] - 571s 571ms/step - loss: 0.2644 - accuracy: 0.8839 - val_loss: 0.2174 - val_accuracy: 0.9096
Epoch 3/30
1000/1000 [==============================] - 574s 574ms/step - loss: 0.2452 - accuracy: 0.8929 - val_loss: 0.2194 - val_accuracy: 0.9064
Epoch 4/30
1000/1000 [==============================] - 574s 574ms/step - loss: 0.2345 - accuracy: 0.9007 - val_loss: 0.2068 - val_accuracy: 0.9130
Epoch 5/30
1000/1000 [==============================] - 592s 592ms/step - loss: 0.2195 - accuracy: 0.9043 - val_loss: 0.2246 - val_accuracy: 0.9028
Epoch 6/30
1000/1000 [==============================] - 605s 605ms/step - loss: 0.2190 - accuracy: 0.9036 - val_loss: 0.2025 - val_accuracy: 0.9148
Epoch 7/30
1000/1000 [==============================] - 604s 604ms/step - loss: 0.2102 - accuracy: 0.9101 - val_